#### Проведем 2 анализа, ABC-анализ и XYZ- анализ.

Набор данных **"Online Retail II"** содержит информацию о продажах для компании в период с 01/12/2009 по 09/12/2011.

Большинство их клиентов — оптовики.

Переменные:

**InvoiceNo**: Номер счета. Уникальный номер для каждой транзакции. Если код начинается с буквы "C", это означает, что транзакция была отменена.

**StockCode**: Код товара. Уникальный номер для каждого продукта.

**Description**: Название продукта.

**Quantity**: Количество товара. Определяет, сколько единиц товара продано в рамках счета.

**InvoiceDate**: Дата и время выставления счета.

**UnitPrice**: Цена за единицу товара.

**CustomerID**: Уникальный идентификатор клиента.

**Country**: Название страны.

In [1]:
import pandas as pd
import numpy as np

C:\Users\dimak\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_excel(r'C:\Users\dimak\OneDrive\Рабочий стол\аналитика\online_retail_II.xlsx')

In [3]:
#Удаляем дубликаты

df = data.drop_duplicates()

In [4]:
# Удаляем отмененные операции

df = df[~df["Invoice"].str.contains("C", na = False)]

In [5]:
#Меняем название стран на привычные

df['Country'].replace({'EIRE':'Ireland', 'RSA':'South Africa'}, inplace=True)

C:\Users\dimak\AppData\Local\Temp\ipykernel_11092\3025835751.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Country'].replace({'EIRE':'Ireland', 'RSA':'South Africa'}, inplace=True)


In [6]:
# Отбираем операции, где количество товаров в чеке больше 0 и где цена товара в чеке больше 0

df = df[(df['Quantity'] > 0) & (df['Price'] > 0)]

In [7]:
#Удаляем пропуски

df = df.dropna()

In [8]:
# Создаем поле "total_price", в котором содердится информация о сумме покупки на котокретный вид товара в чеке

df['total_price'] = df['Price']*df['Quantity']

### ABC - анализ

In [9]:
abc = pd.DataFrame(df.groupby('Customer ID')['total_price'].sum()).reset_index()
abc['percentage'] = abc['total_price']/abc['total_price'].sum()*100
abc = abc.sort_values(by = 'percentage', ascending = False)

In [10]:
def categorize_clients(data):
    total_revenue = abc["total_price"].sum()
    threshold_80 = 0.8 * total_revenue
    threshold_95 = 0.95 * total_revenue

    cumulative_sum = 0
    categories = []
    
    for _, row in abc.iterrows():
        cumulative_sum += row["total_price"]
        if cumulative_sum <= threshold_80:
            categories.append("A")
        elif cumulative_sum <= threshold_95:
            categories.append("B")
        else:
            categories.append("C")
    
    abc["abc"] = categories
    
    return abc

In [11]:
result_df = categorize_clients(abc)


In [12]:
result_df['abc'].value_counts()

abc
C    1717
B    1424
A    1171
Name: count, dtype: int64

**A-клиенты (80%)** – самые важные клиенты, обеспечивающие основную часть дохода. Их необходимо удерживать, предлагать премиальный сервис, персонализированные предложения и бонусные программы.

**B-клиенты (15%)** – клиенты среднего уровня, которые имеют потенциал для роста. Можно стимулировать их частоту покупок с помощью скидок, специальных акций и программ лояльности.

**C-клиенты (5%)** – наименее значимые с точки зрения выручки. Они могут быть разовыми покупателями или случайными клиентами. Фокусироваться на них не стоит, но можно попробовать увеличить их вовлеченность с помощью маркетинговых стратегий.


### XYZ - анализ

In [13]:
xyz = pd.DataFrame(df.groupby('Customer ID')['total_price'].sum()).reset_index()

In [14]:
def xyz_analysis(data):

    mean_value = xyz["total_price"].mean()
    std_dev = xyz["total_price"].std()
    
    xyz["коэффициент вариации"] =  xyz["total_price"].apply(lambda x: (x - mean_value) / std_dev if std_dev != 0 else 0)
    
    conditions = [
        xyz["коэффициент вариации"].abs() < 0.5,
        (xyz["коэффициент вариации"].abs() >= 0.5) & (xyz["коэффициент вариации"].abs() < 1),
        xyz["коэффициент вариации"].abs() >= 1
    ]
    categories = ["X", "Y", "Z"]
    
    xyz["xyz"] = pd.cut(xyz["коэффициент вариации"], bins=[-float('inf'), -0.1, 0.5, float('inf')], labels=["Z", "Y", "X"])
    
    return xyz

In [15]:
result_xyz = xyz_analysis(xyz)

In [16]:
result_xyz['xyz'].value_counts()

xyz
Z    2758
Y    1348
X     206
Name: count, dtype: int64

**X-клиенты** – основа стабильной выручки, их важно удерживать.

**Y-клиенты** – требуют более детального анализа, возможно, им можно предложить персонализированные акции для повышения лояльности.

**Z-клиенты** – сложно прогнозируемая группа, их можно рассматривать как разовые покупки или нестабильный источник дохода.

